In [ ]:
%cd /content/
# 把資料生成工具 clone 下來
!git clone https://github.com/rachellin0105/Single_char_image_generator.git
%cd Single_char_image_generator

In [ ]:
# Single_char_image_generator/chars.txt 是字典，預設有102字，可以在上面增減字。這邊因為是示範，我們只留前10個字。
!head -n 40 chars.txt > temp.txt
!mv temp.txt chars.txt

In [ ]:
# 安裝它需要的套件
!python -m pip install -r requirements.txt

# 用一行指令執行生成 
!python OCR_image_generator_single_ch.py --num_per_word=10

In [ ]:
!git remote add Edge_AI https://github.com/hsyi123/Edge_AI.git
!git fetch Edge_AI
!git checkout Edge_AI/main checkpoint.zip
!unzip checkpoint.zip
!rm checkpoint.zip

In [ ]:
from PIL import Image

import torch
import torchvision.models as models
import torchvision.transforms as transforms

save_path = './checkpoint.pt'

# 创建一个与原始模型相同结构的模型实例
net = models.resnet18(num_classes=40)

# 加载保存的权重
checkpoint = torch.load(save_path, map_location='cpu')
net.load_state_dict(checkpoint['model_state_dict'])

# 设置模型为评估模式
net.eval()

# 這裡可以自己修改成其他要推理的圖
image_path = './output/img_0000000.jpg'
image = Image.open(image_path)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_image = transform(image).unsqueeze(0)

# 使用模型进行推理
with torch.no_grad():
    outputs = net(input_image)

# 获取预测结果
_, predicted = torch.max(outputs, 1)
prediction = predicted.item()

# 输出预测结果
print('Prediction:', prediction)

In [ ]:
import matplotlib.pyplot as plt

# 定义类别标签映射
class_labels = ['肉','古','幼','酥','成','傢','婦','汎','貨','理','男','大','老','樹','民','鴻','禾','髮','酒','麗','鹽','容','由','寵','中','速','食','汽','子','院','批','洗','素','我','快','雞','出','動','品','活']  # 替换为你的实际类别标签

# 获取预测结果
_, predicted = torch.max(outputs, 1)
prediction = predicted.item()

# 根据类别索引获取类别标签
predicted_label = class_labels[prediction-1]

image = Image.open(image_path)

plt.imshow(image)
plt.axis('off')
plt.show()
# 输出预测结果
print('Prediction:', predicted_label)